In [1]:
#import pandas as pd
#census_train = pd.read_csv('census_train.csv')
#census_test = pd.read_csv('census_test.csv')

Pre-process census_train and census_test to get X_train, y_train, X_test and y_test<br/>
Question 2 continues here.

# Q3

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model, tree, model_selection
import numpy as np
from sklearn import tree, model_selection
from sklearn import tree, ensemble, model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
from math import sqrt 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
pd.set_option("display.max_columns", None)


In [3]:
taxi_train=pd.read_csv('taxi-train.csv',dtype={'vendor_id':str,'pickup_datetime':str,'pickup_longitude':float,
                                                'dropoff_datetime':str,
                                                'pickup_latitude':float,'dropoff_longitude':float,'dropoff_latitude':float,
                                              'rate_code':float, 'passenger_count':int,'trip_distance':float,
                                                'payment_type':str,'tip_paid':int,'fare_amount':float,'tip_amount':float})

In [4]:
#drop column with null value count beyond  50% .
thresh = len(taxi_train) * .5
taxi_train.dropna(thresh = thresh, axis = 1, inplace = True)

In [5]:
#Replace all NaN values to the average value of 'rate_code' column
taxi_train['rate_code'].fillna(taxi_train['rate_code'].mean(),inplace=True)

In [6]:
#Calculate the time period spend in taxi In seconds.
taxi_train['pickup_datetime']=pd.to_datetime(taxi_train['pickup_datetime'],format = '%Y-%m-%d %H:%M:%S+00:00')
taxi_train['dropoff_datetime']=pd.to_datetime(taxi_train['dropoff_datetime'],format = '%Y-%m-%d %H:%M:%S+00:00')
taxi_train['take_time_second']=(taxi_train['dropoff_datetime']-taxi_train['pickup_datetime']).dt.total_seconds()

In [7]:
#Calulate the great circle distance between two points on the earth 
def circle_distance(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [8]:
# Calculate the distance by longitude and latitude between two points.
taxi_train['geographical distance']= circle_distance(taxi_train['pickup_longitude'],taxi_train['pickup_latitude'],taxi_train['dropoff_longitude'],taxi_train['dropoff_latitude'])

In [9]:
# Use one-hot encoding for 'vendor_id' column
taxi_train= pd.get_dummies(taxi_train, columns=['vendor_id'])

In [10]:
#some categories number in 'payment_type' is very small , we will combine the small number value.
taxi_train['payment_type'].value_counts()

CSH    545180
CRD    434636
CAS     33097
Cas     27046
Cre     26436
CRE      2856
NOC      1142
UNK      1035
DIS       300
No        131
Dis        28
NA         23
Name: payment_type, dtype: int64

In [11]:
#Combine the small number categories to one category ,then use one-hot encoding for 'payment_type' column
payment_list=['CSH','CRD','CAS','Cas','Cre']
taxi_train['payment_type'] =taxi_train['payment_type'].apply(lambda x: 'OTHER' if x not in payment_list else x)
taxi_train= pd.get_dummies(taxi_train, columns=['payment_type'])

In [12]:
#Drop features that we will not used in traning model
taxi_train.drop(labels=['pickup_datetime','dropoff_datetime','pickup_longitude',
                        'pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

In [13]:
taxi_train

,rate_code,passenger_count,trip_distance,fare_amount,tip_amount,tip_paid,take_time_second,geographical distance,vendor_id_CMT,vendor_id_DDS,vendor_id_VTS,payment_type_CAS,payment_type_CRD,payment_type_CSH,payment_type_Cas,payment_type_Cre,payment_type_OTHER
0,1.0,1,0.80,6.5,0.00,0,445.0,0.883927,1,0,0,0,0,1,0,0,0
1,1.0,1,1.50,6.9,1.88,1,610.0,1.764546,1,0,0,0,0,0,0,1,0
2,1.0,1,1.60,6.9,1.00,1,553.0,2.331919,1,0,0,0,0,0,0,1,0
3,1.0,1,1.70,9.7,1.94,1,960.0,1.892764,0,0,1,0,1,0,0,0,0
4,1.0,2,1.36,6.9,0.00,0,600.0,1.491200,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071905,1.0,1,0.50,5.5,0.00,0,344.0,0.831373,1,0,0,0,0,1,0,0,0
1071906,1.0,1,1.29,8.5,0.00,0,600.0,1.464368,0,0,1,0,1,0,0,0,0
1071907,1.0,1,2.80,8.9,1.88,1,590.0,3.019187,1,0,0,0,1,0,0,0,0
1071908,1.0,1,0.50,4.5,0.00,0,264.0,0.584307,1,0,0,0,0,1,0,0,0


In [14]:
select_column=[0,1,2,5,6,7,8,9,10,11,12,13,14,15,16]

In [15]:
#Get train,validation and test data set , and target value only choose 'fare_amount'.
X_train_fare, X_test_fare, y_train_fare, y_test_fare= train_test_split(taxi_train.iloc[:,select_column], taxi_train.iloc[:,3], test_size=0.2, random_state=1)

X_train_fare, X_val_fare, y_train_fare, y_val_fare= train_test_split(X_train_fare, y_train_fare, test_size=0.25, random_state=1)

In [16]:
#Get train,validation and test data set , and  target value only choose 'tip_amount'.
X_train_tip, X_test_tip, y_train_tip, y_test_tip= train_test_split(taxi_train.iloc[:,select_column], taxi_train.iloc[:,4], test_size=0.2, random_state=1)

X_train_tip, X_val_tip, y_train_tip, y_val_tip= train_test_split(X_train_tip, y_train_tip, test_size=0.25, random_state=1)

# Model of RandomForestRegressor 

In [17]:
from sklearn.ensemble import RandomForestRegressor
param_RF={'n_estimators' : range(10,300,20),
                     'min_samples_split': range(10,300,20),
                     'max_features':range(5,15,2),
                     'min_samples_leaf':range(10,200,20)
                    }

rf_cv=RandomizedSearchCV(RandomForestRegressor(),param_RF,random_state = 1,n_iter=10,refit=True,verbose = 3,n_jobs=-1,cv=5)

##  To predicting target variable 'fare_amount ' with RandomForstRegressor

In [19]:
rf_cv.fit(X_train_fare,y_train_fare)
print('Best parameter for fare_amount in RandomForestRegressor: ',rf_cv.best_params_)


SyntaxError: invalid character in identifier (<ipython-input-19-902ec7e334d6>, line 1)

In [99]:
#Testing the model trained my training data set accuracy by validation data set with evaluation metrics as RMSE. 
predict_val_fare=rf_cv.predict(X_val_fare)
RMSE_RF_val_fare = sqrt(mean_squared_error(y_val_fare,predict_val_fare))
predict_test_fare=rf_cv.predict(X_test_fare)
RMSE_RF_test_fare = sqrt(mean_squared_error(y_test_fare,predict_test_fare))
print('RMSE for validation data set for target variable fare_amount in RandomForestRegressor model: ',RMSE_RF_val_fare)
print('RMSE for test data set with target variable fare_amount in RandomForestRegressor model: ',RMSE_RF_test_fare)

RMSE for validation data set for target variable fare_amount in RandomForestRegressor model:  1.2140830025925857
RMSE for test data set with target variable fare_amount in RandomForestRegressor model:  1.256915143388508


In [100]:
#Use the best parameter to build a RandomForestRegressor and fit with test data to get the feature importance 
rf_fare=RandomForestRegressor(n_estimators=190,min_samples_split=190,max_features=5,min_samples_leaf=10,random_state = 1)
rf_fare.fit(X_test_fare,y_test_fare)
print('Feature importance for target variable fare_amount in RandomForestRegressor',rf_fare.feature_importances_)

Feature importance for target variable fare_amount in RandomForestRegressor [1.38920371e-02 2.93218477e-04 3.09064765e-01 1.42533400e-03
 4.89069212e-01 1.78444002e-01 8.19404902e-05 2.10285670e-04
 8.19119429e-05 4.50973496e-04 4.95535238e-03 1.26533301e-03
 3.04128676e-04 4.37944746e-04 2.35610528e-05]


### Most importance of 5 parameters for fare_amount:     'rate_code' ,'passenger_count	','take_time_second','geographical distance','tip_paid'.

##  To predicting target variable 'tip_amount ' with RandomForstRegressor

In [103]:
rf_cv.fit(X_train_tip,y_train_tip)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'max_features': range(5, 15, 2),
                                        'min_samples_leaf': range(10, 200, 20),
                                        'min_samples_split': range(10, 300, 20),
                                        'n_estimators': range(10, 300, 20)},
                   random_state=1, verbose=3)

In [104]:
#Testing the model trained my training data set accuracy by validation data set with evaluation metrics as RMSE. 
predict_val_tip=rf_cv.predict(X_val_tip)
RMSE_RF_val_tip = sqrt(mean_squared_error(y_val_tip,predict_val_tip ))
predict_test_tip=rf_cv.predict(X_test_tip)
RMSE_RF_test_tip = sqrt(mean_squared_error(y_test_tip,predict_test_tip))
print('RMSE for validation data set for target variable tip_amount in RandomForestRegressor model: ',RMSE_RF_val_tip)
print('RMSE for test data set with target variable tip_amount in RandomForestRegressor model: ',RMSE_RF_test_tip)
print('Best parameter for tip_amount in RandomForestRegressor: ',rf_cv.best_params_)

RMSE for validation data set for target variable tip_amount in RandomForestRegressor model:  0.5543038138024592
RMSE for test data set with target variable tip_amount in RandomForestRegressor model:  0.6392389685359925
Best parameter for tip_amount in RandomForestRegressor:  {'n_estimators': 190, 'min_samples_split': 190, 'min_samples_leaf': 10, 'max_features': 5}


In [105]:
#Use the best parameter to build a RandomForestRegressor and fit with test data to get the feature importance 
rf_tip=RandomForestRegressor(n_estimators=190,min_samples_split=190,max_features=5, 
                             min_samples_leaf=10,random_state = 1)
rf_tip.fit(X_test_tip,y_test_tip)
print('Feature importance for target variable tip_amount in RandomForestRegressor',rf_tip.feature_importances_)

Feature importance for target variable tip_amount in RandomForestRegressor [4.32788034e-03 6.40951793e-04 6.09280463e-02 4.16372621e-01
 8.31439820e-02 2.73993717e-02 4.05647564e-04 6.45529268e-05
 3.40528960e-04 4.18228318e-03 2.55841494e-01 1.33933941e-01
 3.08851102e-03 8.75626834e-03 5.73920223e-04]


###  Most importance of 5 parameters for tip_amount: 'take_time_second','geographical distance', 'rate_code','passenger_count', 'tip_paid'.

# Model of  AdaBoostRegressor

In [130]:
from sklearn.ensemble import AdaBoostRegressor

param_Ada={'n_estimators' : range(10,150,20),
           'learning_rate':[0.0001,0.001,0.01,0.1,1,10,100],
           'loss':['linear', 'square', 'exponential']
                    }
Ada_cv=RandomizedSearchCV(AdaBoostRegressor(),param_Ada,random_state = 1,n_iter=10,refit=True,verbose = 3,n_jobs=-1,cv=5)

## To predicting target variable 'fare_amount ' with AdaBoostRegressor

In [131]:
Ada_cv.fit(X_train_fare,y_train_fare)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=AdaBoostRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1, 10, 100],
                                        'loss': ['linear', 'square',
                                                 'exponential'],
                                        'n_estimators': range(10, 150, 20)},
                   random_state=1, verbose=3)

In [132]:
#Testing the model trained my training data set accuracy by validation data set with evaluation metrics as RMSE. 
predict_val_fare=Ada_cv.predict(X_val_fare)
RMSE_Ada_val_fare = sqrt(mean_squared_error(y_val_fare,predict_val_fare))
predict_test_fare=Ada_cv.predict(X_test_fare)
RMSE_Ada_test_fare = sqrt(mean_squared_error(y_test_fare,predict_test_fare))
print('RMSE for validation data set for target variable fare_amount in AdaBoostRegressor model: ',RMSE_Ada_val_fare)
print('RMSE for test data set with target variable fare_amount in AdaBoostRegressor model: ',RMSE_Ada_test_fare)
print('Best parameter for fare_amount in AdaBoostRegressor: ',Ada_cv.best_params_)

RMSE for validation data set for target variable fare_amount in AdaBoostRegressor model:  1.8397551952148394
RMSE for test data set with target variable fare_amount in AdaBoostRegressor model:  1.885196487215237
Best parameter for fare_amount in AdaBoostRegressor:  {'n_estimators': 70, 'loss': 'exponential', 'learning_rate': 0.01}


In [172]:
#Use the best parameter to build a RandomForestRegressor and fit with test data to get the feature importance 
Ada_fare=AdaBoostRegressor(n_estimators=70,learning_rate= 0.01,loss='exponential' ,random_state = 1)
Ada_fare.fit(X_test_fare,y_test_fare)
print('Feature importance for target variable fare_amount in RandomForestRegressor',Ada_fare.feature_importances_)

Feature importance for target variable fare_amount in RandomForestRegressor [0.01254746 0.         0.14189514 0.         0.8455574  0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]


###  Most importance of 5 features for fare_amount: 'trip_distance','passenger_count','rate_code','take_time_second','tip_paid'

## To predicting target variable 'tip_amount ' with AdaBoostRegressor

In [133]:
Ada_cv.fit(X_train_tip,y_train_tip)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=AdaBoostRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1, 10, 100],
                                        'loss': ['linear', 'square',
                                                 'exponential'],
                                        'n_estimators': range(10, 150, 20)},
                   random_state=1, verbose=3)

In [134]:
#Testing the model trained my training data set accuracy by validation data set with evaluation metrics as RMSE. 
predict_val_tip=Ada_cv.predict(X_val_tip)
RMSE_Ada_val_tip = sqrt(mean_squared_error(y_val_tip,predict_val_tip ))
predict_test_tip=Ada_cv.predict(X_test_tip)
RMSE_Ada_test_tip = sqrt(mean_squared_error(y_test_tip,predict_test_tip))
print('RMSE for validation data set for target variable tip_amount in AdaBoostRegressor model: ',RMSE_Ada_val_tip)
print('RMSE for test data set with target variable tip_amount in AdaBoostRegressor model: ',RMSE_Ada_test_tip)
print('Best parameter for tip_amount in AdaBoostRegressor: ',Ada_cv.best_params_)

RMSE for validation data set for target variable tip_amount in AdaBoostRegressor model:  0.5858971543296739
RMSE for test data set with target variable tip_amount in AdaBoostRegressor model:  0.6652876056831685
Best parameter for tip_amount in AdaBoostRegressor:  {'n_estimators': 90, 'loss': 'square', 'learning_rate': 0.01}


In [173]:
#Use the best parameter to build a RandomForestRegressor and fit with test data to get the feature importance
Ada_tip=AdaBoostRegressor(n_estimators=90,loss='square',learning_rate=0.01,random_state = 1)
Ada_tip.fit(X_test_fare,y_test_fare)
print('Feature importance for target variable tip_amount in AdaBoostRegressor',Ada_tip.feature_importances_)

Feature importance for target variable tip_amount in AdaBoostRegressor [6.00629368e-02 1.12548697e-03 1.31060215e-01 1.85619441e-03
 7.85990954e-01 1.47706741e-02 0.00000000e+00 0.00000000e+00
 1.80661397e-04 0.00000000e+00 4.95287726e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]


## Most importance of 5 features for tip_amount: 'take_time_second','trip_distance','rate_code','geographical_distance'，'passenger_count'

# Model of GradientBoostingRegressor

In [136]:
from sklearn.ensemble import GradientBoostingRegressor

param_Gra={'n_estimators' : range(10,151,20),
           'learning_rate':[0.0001,0.001,0.01,0.1,1,10,100],
           'loss':['ls', 'lad', 'huber','quantile'],
          'min_samples_split': range(10,300,20),
             'max_features':range(5,15,2),
             'min_samples_leaf':range(10,200,20)
                    }
Gra_cv=RandomizedSearchCV(GradientBoostingRegressor(),param_Gra,random_state = 1,n_iter=15,refit=True,verbose = 3,n_jobs=-1,cv=5)

## To predicting target variable 'fare_amount ' with GradientBoostingRegressor

In [137]:
Gra_cv.fit(X_train_fare,y_train_fare)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


C:\Users\28491\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [-4.13532523e+209  1.56961231e-003  3.44500541e-001  9.15489157e-001
  9.27223462e-001  7.37036423e-001 -1.57447579e+000 -3.48277570e+119
 -8.40235021e+285  3.26582647e-002             -inf -1.54439463e+000
 -1.86769771e+199  6.98072096e-001 -2.73519788e+171]
  warnings.warn(
C:\Users\28491\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:925: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -
C:\Users\28491\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:925: RuntimeWarning: overflow encountered in square
  array_stds = np.sqrt(np.average((array -


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=15,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1, 10, 100],
                                        'loss': ['ls', 'lad', 'huber',
                                                 'quantile'],
                                        'max_features': range(5, 15, 2),
                                        'min_samples_leaf': range(10, 200, 20),
                                        'min_samples_split': range(10, 300, 20),
                                        'n_estimators': range(10, 151, 20)},
                   random_state=1, verbose=3)

In [138]:
#Testing the model trained my training data set accuracy by validation data set with evaluation metrics as RMSE. 
predict_val_fare=Gra_cv.predict(X_val_fare)
RMSE_Gra_val_fare = sqrt(mean_squared_error(y_val_fare,predict_val_fare))
predict_test_fare=Gra_cv.predict(X_test_fare)
RMSE_Gra_test_fare = sqrt(mean_squared_error(y_test_fare,predict_test_fare))
print('RMSE for validation data set for target variable fare_amount in GradientBoostingRegressor model: ',RMSE_Gra_val_fare)
print('RMSE for test data set with target variable fare_amount in GradientBoostingRegressor model: ',RMSE_Gra_test_fare)
print('Best parameter for fare_amount in GradientBoostingRegressor: ',Gra_cv.best_params_)

RMSE for validation data set for target variable fare_amount in GradientBoostingRegressor model:  1.0823883339147273
RMSE for test data set with target variable fare_amount in GradientBoostingRegressor model:  1.1243522795156162
Best parameter for fare_amount in GradientBoostingRegressor:  {'n_estimators': 110, 'min_samples_split': 70, 'min_samples_leaf': 30, 'max_features': 7, 'loss': 'ls', 'learning_rate': 0.1}


In [146]:
#Use the best parameter to build a RandomForestRegressor and fit with test data to get the feature importance 
Gra_fare=GradientBoostingRegressor(n_estimators=110 ,min_samples_split=70 ,max_features= 7,
                               min_samples_leaf=30 ,random_state = 1)
Gra_fare.fit(X_test_fare,y_test_fare)
print('Feature importance for target variable fare_amount in GradientBoostingRegressor',Gra_fare.feature_importances_)

Feature importance for target variable fare_amount in GradientBoostingRegressor [3.71072776e-02 1.33184668e-04 3.06539373e-01 7.57890875e-05
 5.20002494e-01 1.29518335e-01 4.50168893e-04 2.68997876e-04
 2.12021096e-04 6.84007580e-04 3.41585806e-03 3.34927064e-04
 5.56082760e-04 6.98863498e-04 2.61964622e-06]


## Most importance of 5 features for fare_amount: 'take_time_second','trip_distance','geographical distance','rate_code','payment_CRD'

## To predicting target variable 'tip_amount ' with GradientBoostingRegressor

In [139]:
Gra_cv.fit(X_train_tip,y_train_tip)


Fitting 5 folds for each of 15 candidates, totalling 75 fits


C:\Users\28491\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [-8.15454645e+203  1.32393148e-003  2.93457205e-001  6.99311133e-001
  7.01147073e-001  4.96654534e-001 -1.38024949e+000 -1.54234152e+011
 -4.42843891e+286 -2.74239616e-001             -inf -1.33289165e+000
 -8.86623384e+199  5.65610087e-001 -6.18144215e+171]
  warnings.warn(
C:\Users\28491\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:925: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -
C:\Users\28491\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:925: RuntimeWarning: overflow encountered in square
  array_stds = np.sqrt(np.average((array -


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=15,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1, 10, 100],
                                        'loss': ['ls', 'lad', 'huber',
                                                 'quantile'],
                                        'max_features': range(5, 15, 2),
                                        'min_samples_leaf': range(10, 200, 20),
                                        'min_samples_split': range(10, 300, 20),
                                        'n_estimators': range(10, 151, 20)},
                   random_state=1, verbose=3)

In [140]:
#Testing the model trained my training data set accuracy by validation data set with evaluation metrics as RMSE. 
predict_val_tip=Gra_cv.predict(X_val_tip)
RMSE_Gra_val_tip = sqrt(mean_squared_error(y_val_tip,predict_val_tip ))
predict_test_tip=Gra_cv.predict(X_test_tip)
RMSE_Gra_test_tip = sqrt(mean_squared_error(y_test_tip,predict_test_tip))
print('RMSE for validation data set for target variable tip_amount in GradientBoostingRegressor model: ',RMSE_Gra_val_tip)
print('RMSE for test data set with target variable tip_amount in GradientBoostingRegressor model: ',RMSE_Gra_test_tip)
print('Best parameter for tip_amount in GradientBoostingRegressor: ',Gra_cv.best_params_)

RMSE for validation data set for target variable tip_amount in GradientBoostingRegressor model:  0.5536392480597323
RMSE for test data set with target variable tip_amount in GradientBoostingRegressor model:  0.6384721447109518
Best parameter for tip_amount in GradientBoostingRegressor:  {'n_estimators': 110, 'min_samples_split': 70, 'min_samples_leaf': 30, 'max_features': 7, 'loss': 'ls', 'learning_rate': 0.1}


In [141]:
#Use the best parameter to build a RandomForestRegressor and fit with test data to get the feature importance
Gra_tip=GradientBoostingRegressor(n_estimators=190,min_samples_split=190,max_features=5,min_samples_leaf=10,random_state = 1)
Gra_tip.fit(X_test_tip,y_test_tip)
print('Feature importance for target variable tip_amount in GradientBoostingRegressor',Gra_tip.feature_importances_)

Feature importance for target variable tip_amount in GradientBoostingRegressor [8.51467713e-03 1.93616796e-05 4.44367664e-02 5.07058245e-01
 9.99257422e-02 1.79487862e-02 1.65483403e-04 3.23097793e-05
 4.70565050e-05 1.44161389e-04 1.21466077e-01 1.97875453e-01
 5.88908646e-05 2.30143009e-03 5.55933993e-06]


## Most importance of 5 features for tip_amount: 'tip_paid','payment_type_CSH','payment_type_CRD','take_time_second','trip_distance'

# For target variable fare_amount

<b>Random Forest</b><br/>
Objective metric on test:  1.256915143388508 <br/>
Parameters: {'n_estimators': 190, 'min_samples_split': 190, 'min_samples_leaf': 10, 'max_features': 5} 


<b>AdaBoost</b><br/>
Objective metric on test:  1.885196487215237 <br/>
Parameters:  {'n_estimators': 70, 'loss': 'exponential', 'learning_rate': 0.01} 

<b>Gradient Boosting</b><br/>
Objective metric on test:  1.1243522795156162<br/>
Parameters:  {'n_estimators': 110, 'min_samples_split': 70, 'min_samples_leaf': 30, 'max_features': 7, 'loss': 'ls', 'learning_rate': 0.1} 

<b>Important Features</b>

| Classifier | RandomForest | AdaBoost | GradientBoosting |
| --- | --- | --- | --- |
| Feature 1 |rate_code|trip_distance |take_time_second  
| Feature 2 |passenger_count|passenger_count|trip_distance
| Feature 3 |take_time_second|rate_code|geographical distance
| Feature 4 |geographical distance|take_time_second|rate_code
| Feature 5 |tip_paid|tip_paid|payment_CRD

## For target variable tip_amount

<b>Random Forest</b><br/>
Objective metric on test: 0.6392389685359925<br/>
Parameters: {'n_estimators': 190, 'min_samples_split': 190, 'min_samples_leaf': 10, 'max_features': 5}


<b>AdaBoost</b><br/>
Objective metric on test: 0.6652876056831685<br/>
Parameters: {'n_estimators': 90, 'loss': 'square', 'learning_rate': 0.01}

<b>Gradient Boosting</b><br/>
Objective metric on test: 0.6384721447109518<br/>
Parameters: {'n_estimators': 110, 'min_samples_split': 70, 'min_samples_leaf': 30, 'max_features': 7, 'loss': 'ls', 'learning_rate': 0.1}

<b>Important Features</b>

| Classifier | RandomForest | AdaBoost | GradientBoosting |
| --- | --- | --- | --- |
| Feature 1 | take_time_second|take_time_second|tip_paid
| Feature 2 |geographical distance|trip_distance|payment_type_CSH
| Feature 3 |rate_code|rate_code|payment_type_CRD|payment_type_CRD
| Feature 4 |passenger_count|geographical_distance|take_time_second
| Feature 5 |tip_paid|passenger_count|trip_distance

In [206]:

taxi_test=pd.read_csv('taxi-test.csv',dtype={'vendor_id':str,'pickup_datetime':str,'pickup_longitude':float,
                                                'dropoff_datetime':str,
                                                'pickup_latitude':float,'dropoff_longitude':float,'dropoff_latitude':float,
                                              'rate_code':float, 'passenger_count':int,'trip_distance':float,
                                                'payment_type':str})

In [207]:
thresh = len(taxi_test) * .5
taxi_test.dropna(thresh = thresh, axis = 1, inplace = True)
#valmean=pd.to_numeric(taxi_train['rate_code'], errors='coerce').mean()
taxi_test['rate_code'].fillna(taxi_test['rate_code'].mean(),inplace=True)


In [208]:
taxi_test['pickup_datetime']=pd.to_datetime(taxi_test['pickup_datetime'],format = '%Y-%m-%d %H:%M:%S+00:00')
taxi_test['dropoff_datetime']=pd.to_datetime(taxi_test['dropoff_datetime'],format = '%Y-%m-%d %H:%M:%S+00:00')
taxi_test['take_time_second']=(taxi_test['dropoff_datetime']-taxi_test['pickup_datetime']).dt.total_seconds()

In [209]:
#Calulate the great circle distance between two points on the earth 
def circle_distance(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [210]:
taxi_test['geographical distance']= circle_distance(taxi_test['pickup_longitude'],taxi_test['pickup_latitude'],taxi_test['dropoff_longitude'],taxi_test['dropoff_latitude'])

In [211]:
taxi_test= pd.get_dummies(taxi_test, columns=['vendor_id'])

In [212]:
taxi_test['payment_type'].value_counts()

CSH    5392
CRD    4460
CAS     344
Cas     302
Cre     272
CRE      42
UNK       8
NOC       7
DIS       1
Name: payment_type, dtype: int64

In [213]:
#Combine the small value to 'OTHER'
payment_list=['CSH','CRD','CAS','Cas','Cre','CRE']
taxi_test['payment_type'] =taxi_test['payment_type'].apply(lambda x: 'OTHER' if x not in payment_list else x)

In [214]:
taxi_test= pd.get_dummies(taxi_test, columns=['payment_type'])

In [215]:
taxi_test.drop(labels=['pickup_datetime','dropoff_datetime','pickup_longitude',
                        'pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

## Accroding the model metrics result above, Gradient Boosting has the best RMSE which is  1.1243522795156162 in predict 'fare_amount' . So, we use GradientBoostingRegressor with parameters {'n_estimators': 110, 'min_samples_split': 70, 'min_samples_leaf': 30, 'max_features': 7, 'loss': 'ls', 'learning_rate': 0.1} to predict 'fare_amount' value.


In [223]:
#Gra_fare is the model trained by taxi-train data set test part with best parameters.
target_fare_amount=Gra_fare.predict(taxi_test)


In [224]:
#Predicted fare_amount in taxi-test dataset
target_fare_amount

array([4.85655745, 4.41725918, 7.37879595, ..., 6.92783764, 4.04101195,
       8.64778862])

## Accroding the model metrics result above, Gradient Boosting has the best RMSE which is   0.6384721447109518 in 'predict tip_amount'. So, we use GradientBoostingRegressor with parameters {'n_estimators': 110, 'min_samples_split': 70, 'min_samples_leaf': 30, 'max_features': 7, 'loss': 'ls', 'learning_rate': 0.1} to predict 'tip_amount' value.

In [225]:
#Gra_tip is the model trained by taxi-train data set test part with best parameters.
target_fare_amount=Gra_tip.predict(taxi_test)

In [226]:
#Predicted tip_amount in taxi-test dataset
target_fare_amount

array([1.41860743, 1.60606063, 1.66796867, ..., 1.78373896, 1.53489041,
       1.71104303])